## Navigate to the Correct Directory

The following code navigates to the dataprocessing directory.

In [1]:
cd ../dataprocessing

/usr/local/google/home/carverforbes/activity-recognition/location/lbs/activity/audioset/dataprocessing


## Call the Import Statements

The following code imports the necessary code to run the code in the rest of this notebook.

In [2]:
# import statements
import audio_processing as ap

import functools
import os
import sys
from absl import logging

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow import feature_column
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import sklearn

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
# tf.keras.backend.set_floatx('float32')

print("Ran the import statements.")

Ran the import statements.


In [3]:
# The directory should be arranged in form:
# .
# ├── audioset_v1_embeddings
# ├── class_labels_indices.csv
# └── Model_on_VGG.ipynb

src_dir = "example_src_dir"
dest_dir = 'example_dest_dir'
path = "audioset_v1_embeddings"
eva = "eval"
bal = "bal_train"
unbal = "unbal_train"

## Feature Extraction
Configure the following parameters to extract the desired features from a specified csv file to a specific destination directory.

In [4]:
# set logging to print logging.INFO logs
logging.set_verbosity(logging.INFO)

In [5]:
def read_tfrecord(filename):
    # Read in a tfrecord file
    # Store information in list of lists
    # Audio_embedding is a list of 10 embeddings, each represents 1 second feature
    #
    # input: str filename
    # output: pandas dataframe with columns:
    #        [str video_id, float start_time, float end_time, list label_index, list embed]
    
    if not filename.endswith('.tfrecord'):
        print("This file is not a .tfrecord file.")
        return
    raw_dataset = tf.data.TFRecordDataset(filename)
    return_df = pd.DataFrame(columns=
                             ['video_id', 'start_time_seconds', 'end_time_seconds', 'labels', 'audio_embedding'])
    for raw_record in raw_dataset:
        cur_record_list = []
        example = tf.train.SequenceExample()
        example.ParseFromString(raw_record.numpy())
        
        cur_record_list.append(example.context.feature['video_id'].bytes_list.value[0].decode("utf-8"))
        cur_record_list.append(example.context.feature['start_time_seconds'].float_list.value[0])
        cur_record_list.append(example.context.feature['end_time_seconds'].float_list.value[0])
        cur_record_list.append(example.context.feature['labels'].int64_list.value)
        
        # Original embeddings are stored in hex format, now convert them to readable int
        embeds = []
        for i in range(len(example.feature_lists.feature_list['audio_embedding'].feature)):
            hexembed = example.feature_lists.feature_list['audio_embedding'].feature[i].bytes_list.value[0].hex()
            arrayembed = [int(hexembed[i:i+2], 16) for i in range(0, len(hexembed), 2)]
            embeds.append(arrayembed)
        cur_record_list.append(embeds)
        
        return_df.loc[len(return_df)] = cur_record_list
    return return_df

In [17]:
def get_vgg_tfrecords(dir_path):
    final_df = pd.DataFrame(
        columns=['video_id', 'start_time_seconds', 'end_time_seconds', 'labels', 'audio_embedding'])
    for file in os.listdir(dir_path):
        if file.endswith(".tfrecord"):
            file_path = os.path.join(dir_path, file)
            df = read_tfrecord(file_path)
            final_df = final_df.append(df, ignore_index=True)
    return final_df

In [18]:
def label_positive_examples(dataframe, positive_labels, csv_path):
    # output set of label ids to use as positive labels
    labels_df = pd.read_csv(csv_path)
    positive_labels = set(positive_labels)
    positive_indices = set()
    for index in labels_df.index:
        if labels_df['display_name'][index] in positive_labels:
            positive_indices.add(index)
    for index in dataframe.index:
        positive = False
        for label in dataframe.labels[index]:
            if label in positive_indices:
                positive = True
                dataframe.labels[index] = 1
                break
        if not positive:
            dataframe.labels[index] = 0

In [19]:
def count_positive(dataframe):
    count = 0
    for index in dataframe.index:
        if dataframe.labels[index] == 1:
            count += 1
    return count

In [28]:
def balance_dataset(dataframe, ratio):
    positive_df = pd.DataFrame(columns=['label', 'feature'])
    negative_df = pd.DataFrame(columns=['label', 'feature'])
    for index in dataframe.index:
        if dataframe.labels[index] == 1:
            temp = pd.DataFrame([[1, dataframe['audio_embedding'][index]]], columns=['label', 'feature'])
            positive_df = positive_df.append(temp, ignore_index=True)
        elif dataframe.labels[index] == 0:
            temp = pd.DataFrame([[0, dataframe['audio_embedding'][index]]], columns=['label', 'feature'])
            negative_df = negative_df.append(temp, ignore_index=True)
    positive_df = positive_df.append(negative_df.sample(len(positive_df) * ratio), ignore_index=True)
    return positive_df

In [36]:
def separate_seconds(dataframe):
    df = pd.DataFrame(columns=['label', 'feature'])
    for index in dataframe.index:
        for arr in dataframe.feature[index]:
            if len(arr) == 128:  
                temp = pd.DataFrame([[dataframe.label[index], arr]], columns=['label', 'feature'])
                df = df.append(temp, ignore_index=True)                
        print(df.index.stop)
    return df

In [22]:
def normalize_features(dataframe):
    for index in dataframe.index:
        arr = np.array(dataframe['feature'][index]) / 255
        dataframe['feature'][index] = arr
    return dataframe

In [23]:
def shuffle_dataframe(dataframe):
    # Shuffle the dataset/dataframe.
    dataframe = dataframe.reindex(np.random.permutation(dataframe.index))
    return dataframe

In [24]:
def dataframe_to_x_y(dataframe):
    X = np.array(dataframe.feature.tolist(), dtype=object)
    y = np.array(dataframe.label.tolist())
    
    # Convert arrays of objects to arrays of floats.
    X = tf.keras.backend.cast_to_floatx(X)
    y = tf.keras.backend.cast_to_floatx(y)
    return X, y

In [30]:
def get_x_y_from_dataset(dataset_path, class_labels_csv_path, label_list, ratio=1):
    print('Getting dataframe from .tfrecord files')
    df = get_vgg_tfrecords(dataset_path)
    print('Setting positive examples to label 1')
    label_positive_examples(df, label_list, class_labels_csv_path)
    print('balancing dataset')
    df = balance_dataset(df, ratio)
    print('shuffling dataset')
    df = shuffle_dataframe(df)
    print('normalizing features')
    df = normalize_features(df)
    print('splitting data into 1 second intervals')
    df = separate_seconds(df)
    X, y = dataframe_to_x_y(df)
    return X, y

In [31]:
def get_x_y_from_dataset_list(dataset_path_list, class_labels_csv_path, label_list, ratio=1):
    length = 0 if dataset_path_list is None else len(dataset_path_list)
    if length <= 0 or length > 3:
        print('Cannot have less than one training set and more than a training, validation, and test set')
        return None
    elif length == 1:
        dataset_path = dataset_path_list[0]
        X, y = get_x_y_from_dataset(dataset_path, class_labels_csv_path, label_list, ratio)
        x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state = 42)
        return x_train, y_train, x_val, y_val
    elif length == 2:
        train_path = dataset_path_list[0]
        val_path = dataset_path_list[1]
        x_train, y_train = get_x_y_from_dataset(train_path, class_labels_csv_path, label_list, ratio)
        x_val, y_val = get_x_y_from_dataset(val_path, class_labels_csv_path, label_list, ratio)
        return x_train, y_train, x_val, y_val
    else:
        train_path = dataset_path_list[0]
        val_path = dataset_path_list[1]
        test_path = dataset_path_list[2]
        x_train, y_train = get_x_y_from_dataset(train_path, class_labels_csv_path, label_list, ratio)
        x_val, y_val = get_x_y_from_dataset(val_path, class_labels_csv_path, label_list, ratio)
        x_test, y_test = get_x_y_from_dataset(test_path, class_labels_csv_path, label_list, ratio)
        return x_train, y_train, x_val, y_val, x_test, y_test

In [37]:
bal_path = os.path.join(src_dir, path, bal)
unbal_path = os.path.join(src_dir, path, unbal)
eva_path = os.path.join(src_dir, path, eva)
class_labels_path = os.path.join(src_dir, 'class_labels_indices.csv')
labels_to_be_positive = ['Gunshot, gunfire', 'Machine gun', 'Fusillade', 'Artillery fire']
# dataset_path_list = [unbal_path, bal_path, eva_path]
dataset_path_list = [bal_path]
ratio = 1

data = get_x_y_from_dataset_list(
    dataset_path_list, class_labels_path, labels_to_be_positive, ratio)
# x_train, y_train, x_val, y_val, x_test, y_test = data
x_train, y_train, x_val, y_val = data

Getting dataframe from .tfrecord files
Setting positive examples to label 1


<ipython-input-18-81db929b1287>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.labels[index] = 0
<ipython-input-18-81db929b1287>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.labels[index] = 1


balancing dataset
shuffling dataset
normalizing features
splitting data into 1 second intervals
10
20
30
40
50
60
70
80
87
97
107
117
127
137
147
157
167
177
187
197
207
217
227
237
247
257
267
277
287
297
304
314
324
334
344
354
364
374
384
394
404
414
424
434
444
454
464
474
484
494
504
514
524
534
544
554
564
574
584
594
604
614
624
634
644
654
664
674
684
694
704
714
724
734
744
754
764
774
784
794
804
814
824
834
844
854
864
874
884
894
904
914
924
934
944
954
964
974
984
987
997
1007
1017
1027
1037
1047
1057
1067
1077
1087
1097
1107
1117
1127
1137
1147
1157
1167
1177
1187
1196
1206
1216
1226
1236
1246
1256
1266
1276
1286
1296
1304
1314
1324
1334
1344
1354
1364
1374
1384
1394
1404
1414
1424
1434
1444
1454
1464
1474
1484
1494
1504
1514
1524
1534
1544
1554
1564
1574
1584
1594
1604
1614
1624
1634
1644
1654
1664
1674
1684
1694
1704
1714
1724
1734
1744
1754
1764
1773
1783
1793
1803
1813
1823
1833
1843
1853
1863
1873
1883
1893
1903
1913
1923
1933
1943
1953
1963
1973
1983
1993
2003
2013


## Building and Training Neural Network

In [16]:
# Define the plotting function.
def plot_curve(epochs, hist, list_of_metrics, path, filename, list_of_hyperparameters):
    """Plot a curve of one or more classification metrics vs. epoch and save it to path."""  
    # list_of_metrics should be one of the names shown in:
    # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics  

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Value")

    for m in list_of_metrics:
        x = hist[m]
        plt.plot(epochs[1:], x[1:], label=m)

    plt.legend()
    
    if not os.path.isdir(path):
        try:
            os.mkdir(path)
        except OSError as error:
            logging.error(error)
            
    path = os.path.join(path, filename)
    
    if not os.path.isdir(path):
        try:
            os.mkdir(path)
        except OSError as error:
            logging.error(error)
        
    list_of_hyperparameters_temp = [str(item) for item in list_of_hyperparameters]
    list_of_metrics_temp = [item if isinstance(item, str) else str(item.name) for item in list_of_metrics]
    filename = '_'.join(list_of_metrics_temp) + '_' + '_'.join(list_of_hyperparameters_temp)
    path = os.path.join(path, filename + '.png')
    plt.savefig(path, bbox_inches='tight')
    
    return plt


print("Defined the plot_curve function.")

Defined the plot_curve function.


## Train 1
Neural Network Type:
* Input Layer: (Number of Nodes)
* Hidden Layer 1:
* Hidden Layer 2: 
* Ouput Layer: 

Hyper-parameters:
* Loss Function: 
* Activation Function: 
* Optimizer Function: 
* Learning Rate: 
* Epochs: 
* Batch_Size: 
* Classification Threshold: 
* Regularization: 
* Regularization Lambda: 

Notes:
* Data was balanced with a 1:20 ratio between positive and negative examples

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics, activation, optimizer, regularization, regularization_lambda):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()

    # Add the input layer of ___ nodes
    model.add(tf.keras.layers.Dense(units=128, input_shape=(128,)))
              
    # Implement ___ regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=128, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden1'))
    
#     # Implement ___ regularization in the second hidden layer.
#     model.add(tf.keras.layers.Dense(units=128, 
#                                   activation=activation,
#                                   kernel_regularizer=regularization(regularization_lambda),
#                                   name='Hidden2'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=optimizer(lr=my_learning_rate),                                                   
                loss=loss,
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.0,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")

In [ ]:
# The following variables are the hyperparameters.
ratio_list = [1, 20, 40]
activation_list = ['elu', 'exponential', 'relu', 'selu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh']
optimizer_list = ['adadelta', 'adagrad', 'adam', 'adamax', 'ftrl', 'nadam', 'rmsprop', 'sgd']
loss = tf.keras.losses.BinaryCrossentropy()
activation = 'relu'
optimizer = tf.keras.optimizers.RMSprop
learning_rate = 0.001
epochs = 50
batch_size = 25
classification_threshold = 0.70
regularization = tf.keras.regularizers.l2
regularization_lambda = 0.001
label_name = "label"
filename = 'balanced_train_segments'
dest_dir = 'example_dest_dir'

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           regularization_lambda,
                           label_name]


# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

# Establish the model's topography.
my_model = create_model(learning_rate, METRICS, activation, optimizer, regularization, regularization_lambda)

# View the model's structure.
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(
    my_model, x_train, y_train, epochs, label_name, batch_size, validation_data=(
        x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, list_of_metrics_to_plot, dest_dir, filename, list_of_hyperparameters)
plot_curve(epochs, hist, ['loss', 'val_loss'], dest_dir, filename, list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

## Train 2
Neural Network Type:
* Input Layer: (Number of Nodes)
* Hidden Layer 1:
* Hidden Layer 2: 
* Ouput Layer: 

Hyper-parameters:
* Loss Function: 
* Activation Function: 
* Optimizer Function: 
* Learning Rate: 
* Epochs: 
* Batch_Size: 
* Classification Threshold: 
* Regularization: 
* Regularization Lambda: 

Notes:
* Data was balanced with a 1:20 ratio between positive and negative examples

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics, activation, optimizer, regularization, regularization_lambda):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()

    # Add the input layer of ___ nodes
    model.add(tf.keras.layers.Dense(units=128, input_shape=(128,)))
              
    # Implement ___ regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=128, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden1'))
    
    # Include a dropout layer.
    model.add(tf.keras.layers.Dropout(0.2))
    
    # Implement ___ regularization in the second hidden layer.
    model.add(tf.keras.layers.Dense(units=128, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden2'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=optimizer(lr=my_learning_rate),                                                   
                loss=loss,
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.0,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")

In [ ]:
# The following variables are the hyperparameters.
ratio_list = [1, 20, 40]
activation_list = ['elu', 'exponential', 'relu', 'selu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh']
optimizer_list = ['adadelta', 'adagrad', 'adam', 'adamax', 'ftrl', 'nadam', 'rmsprop', 'sgd']
loss = tf.keras.losses.BinaryCrossentropy()
activation = 'relu'
optimizer = tf.keras.optimizers.RMSprop
learning_rate = 0.001
epochs = 100
batch_size = 25
classification_threshold = 0.70
regularization = tf.keras.regularizers.l2
regularization_lambda = 0.001
label_name = "label"
filename = 'balanced_train_segments'
dest_dir = 'example_dest_dir'

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           regularization_lambda,
                           label_name]


# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]


# Establish the model's topography.
my_model = create_model(learning_rate, METRICS, activation, optimizer, regularization, regularization_lambda)

# View the model's structure.
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(
    my_model, x_train, y_train, epochs, label_name, batch_size, validation_data=(
        x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, list_of_metrics_to_plot, dest_dir, filename, list_of_hyperparameters)
plot_curve(epochs, hist, ['loss', 'val_loss'], dest_dir, filename, list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

## Train 3
Neural Network Type:
* Input Layer: (Number of Nodes)
* Hidden Layer 1:
* Hidden Layer 2: 
* Ouput Layer: 

Hyper-parameters:
* Loss Function: 
* Activation Function: 
* Optimizer Function: 
* Learning Rate: 
* Epochs: 
* Batch_Size: 
* Classification Threshold: 
* Regularization: 
* Regularization Lambda: 

Notes:
* Data was balanced with a 1: ratio between positive and negative examples

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics, activation, optimizer, regularization, regularization_lambda):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()

    # Add the input layer of ___ nodes
    model.add(tf.keras.layers.Dense(units=128, input_shape=(128,)))
              
    # Implement ___ regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=128, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden1'))
    
#     # Implement ___ regularization in the second hidden layer.
#     model.add(tf.keras.layers.Dense(units=128, 
#                                   activation=activation,
#                                   kernel_regularizer=regularization(regularization_lambda),
#                                   name='Hidden2'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=optimizer(lr=my_learning_rate),                                                   
                loss=loss,
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.0,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")

In [ ]:
# The following variables are the hyperparameters.
ratio_list = [1, 20, 40]
activation_list = ['elu', 'exponential', 'relu', 'selu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh']
optimizer_list = ['adadelta', 'adagrad', 'adam', 'adamax', 'ftrl', 'nadam', 'rmsprop', 'sgd']
loss = tf.keras.losses.BinaryCrossentropy()
activation = 'relu'
optimizer = tf.keras.optimizers.RMSprop
learning_rate = 0.001
epochs = 100
batch_size = 25
classification_threshold = 0.70
regularization = tf.keras.regularizers.l2
regularization_lambda = 0.001
label_name = "label"
filename = 'balanced_train_segments'
dest_dir = 'example_dest_dir'

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           regularization_lambda,
                           label_name]


# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

# Establish the model's topography.
my_model = create_model(
    learning_rate, METRICS, activation, optimizer, regularization, regularization_lambda)

# View the model's structure.
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(
    my_model, x_train, y_train, epochs, label_name, batch_size, validation_data=(
        x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 
                           'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(
    epochs, hist, list_of_metrics_to_plot, dest_dir, filename, list_of_hyperparameters)
plot_curve(
    epochs, hist, ['loss', 'val_loss'], dest_dir, filename, list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])